In [6]:
import torch
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import torch.nn as nn
import nemo.collections.asr as nemo_asr


[NeMo W 2024-10-12 10:34:55 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-12 10:34:55 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:265: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-10-12 10:34:55 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:325: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-12 10:34:55 nemo_loggin

# Duration Augmented Content Vectors 

In [2]:
def cosine_similarity(v1, v2):
    return F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0), dim=-1).item()

In [1]:
def group_similar_vectors(vectors, threshold, vector_duration):
    grouped_vectors = []
    durations = []

    current_group = [vectors[:, 0]]  # Start with the first vector (slice across batch dimension)
    current_duration = vector_duration

    for i in range(1, vectors.shape[-1]):
        sim = cosine_similarity(vectors[:, i], vectors[:, i - 1])
        # print(sim)
        # If cosine similarity is above threshold, group the vectors
        if sim > threshold:
            # print("Exceeded Threshold")
            current_group.append(vectors[:, i])
            current_duration += vector_duration
        else:
            # Compute the average of the current group and save it
            averaged_vector = torch.mean(torch.stack(current_group, dim=-1), dim=-1)
            grouped_vectors.append(averaged_vector)
            durations.append(current_duration)

            # Start a new group
            current_group = [vectors[:, i]]
            current_duration = vector_duration

    # Append the last group
    if current_group:
        averaged_vector = torch.mean(torch.stack(current_group, dim=-1), dim=-1)
        grouped_vectors.append(averaged_vector)
        durations.append(current_duration)

    return torch.stack(grouped_vectors, dim=-1), durations

# Example usage
# Assume `content_embeddings` is the input tensor of shape [1, 256, num_content_vectors]
# `T`: Cosine similarity threshold
# `vector_duration`: The duration of a single vector (calculated as ~46.44 milliseconds)

def duration_augmented_representation(content_embeddings, T=0.925, vector_duration=46.44):
    # Remove the batch dimension for processing
    content_vectors = content_embeddings.squeeze(0)  # Shape: [256, num_content_vectors]

    grouped_vectors, new_durations = group_similar_vectors(content_vectors, T, vector_duration)

    # Convert durations to seconds if needed
    new_durations_in_seconds = [d / 1000 for d in new_durations]

    # Add the batch dimension back to the output
    grouped_vectors = grouped_vectors.unsqueeze(0)  # Shape: [1, 256, num_grouped_vectors]

    return grouped_vectors, new_durations_in_seconds

In [7]:
# class ConformerEncoder256(nn.Module):
#     def __init__(self, original_encoder):
#         super(ConformerEncoder256, self).__init__()
#         self.encoder = original_encoder
#         # Add a 1D convolution with stride to downsample to 256
#         self.downsample = nn.Conv1d(in_channels=512, out_channels=256, kernel_size=1, stride=2) # Example

#     def forward(self, audio_signal, length):
#         # Pass through original encoder
#         embeddings, lengths = self.encoder(audio_signal=audio_signal, length=length)
#         # Reduce the embedding size
#         embeddings = self.downsample(embeddings)
#         return embeddings, lengths

In [8]:
# audio_path = '/home/keagan/Documents/projects/SelfVC/data/archive/14/208/14_208_000001_000000.wav'
# conformer_encoder_256 = torch.load('/home/keagan/Documents/projects/SelfVC/models/conformer_encoder_v2.pth')
# mel_transform = T.MelSpectrogram(
#             sample_rate=22050,
#             n_mels=80,
#             n_fft=1024,
#             hop_length=256,
#             win_length=1024,
#         )
# waveform, sr = torchaudio.load(audio_path)
# mel_spectrogram = mel_transform(waveform)

[NeMo W 2024-10-12 10:35:10 nemo_logging:349] /tmp/ipykernel_458095/3116426420.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      conformer_encoder_256 = torch.load('/h

In [34]:
# output_vectors = torch.randn(1,256,100)  # Replace with actual model output

# T = 0.9  # Cosine similarity threshold
# grouped_vectors, new_durations = duration_augmented_representation(mel_spectrogram, T)


0.6877931952476501
0.9050092101097107
0.9989377856254578
0.7195181846618652
0.32542920112609863
0.9276144504547119
0.8478531837463379
0.9701856374740601
0.5990466475486755
0.940853476524353
0.9752126336097717
0.995766818523407
0.9964480996131897
0.9980087280273438
0.9983047842979431
0.9794524908065796
0.8990389704704285
0.9281880855560303
0.9794193506240845
0.9871618747711182
0.9896873831748962
0.9884335994720459
0.9945268034934998
0.9794251322746277
0.9752745628356934
0.971794068813324
0.9383509159088135
0.9704168438911438
0.9689639210700989
0.9360604882240295
0.9477853178977966
0.9288196563720703
0.9143929481506348
0.8610033392906189
0.8551521897315979
0.7321525812149048
0.76909339427948
0.8111119270324707
0.9545629024505615
0.6977664828300476
0.8729537129402161
0.895615816116333
0.877685010433197
0.9423441290855408
0.9776462912559509
0.9057173728942871
0.8589544296264648
0.9785416126251221
0.9564515948295593
0.6259208917617798
0.8280013203620911
0.9967959523200989
0.9915792942047119